# Functions

In [1]:
import os
import math
import numpy as np
from matplotlib.pylab import imshow, figure, show, savefig, title
import matplotlib.cm as cm
from skimage import io
import skimage.draw as draw
import skimage.feature as feature
import skimage.filters as filters
import scipy.ndimage as nd

# Sobel Edge Detector
def Sobel(img, thresh):
    sobel = filters.sobel(img)
    if thresh == "default":
        thresh = np.absolute(sobel).max()*0.2
    for i in range(len(sobel)):
        for j in range(len(sobel[i])):
            if sobel[i][j] > thresh:
                sobel[i][j] = 1
            else:
                sobel[i][j] = 0
    return sobel

# given a fixed radius, this function maps an edge map to an accumulator (Hough Space)
def HoughSpace(edge_map, radius, usegradient, quant):
    h = len(edge_map)
    w = len(edge_map[0])
    
    # storing coordinates of all edges
    x_edge = []
    y_edge = []
    for y in range(h):
        for x in range(w):
            if edge_map[y][x] == 1:
                x_edge.append(x)
                y_edge.append(y)
                
    # initialising accumulator
    accumulator = np.zeros((h, w))
    
    # mapping edge map to accumulator using the formula: a = x +- sqrt(r^2-(y-b)^2)
    num_edges = len(x_edge) 
    r2 = radius*radius
    for b in range(0,h,quant):
        for n in range(num_edges):
            temp = r2 - (y_edge[n] - b)**2 # r^2-(y-b)^2
            if temp >= 0: #account for negative square roots
                a1 = int(x_edge[n] - math.sqrt(temp))
                a2 = int(x_edge[n] + math.sqrt(temp))
                for n in range(quant):
                    if a1 + n >= 0 and a1 + n <= w-1:
                        accumulator[b][a1+n] += 1
                    elif a2 + n >= 0 and a2 + n <= w-1:
                        accumulator[b][a2+n] += 1
                    
    return accumulator


# this function computes the coordinates of circle centres, given the edge detector type, circle radius and whether gradient is used. It also saves its edge map and Hough Space for reference.
def DetectCircles(img, radius, usegradient, quant, filename, edge_type):
    # If Canny Edge Detector is used
    if edge_type == "canny":
        if usegradient == "usegradient":
            edge_map = feature.canny(img)
        else:
            edge_map = feature.canny(img, low_threshold =0, high_threshold = 0) #threshold = 0 as magnitude of gradient not used
    
    elif edge_type == "sobel":
        if usegradient == "usegradient":
            edge_map = Sobel(img, "default") #threshold = 0 as magnitude of gradient not used
        else:
            edge_map = Sobel(img, 0.0) 
    
    # Drawing Edge Map
    imshow(edge_map, cmap=cm.gray)
    title("Edge Map")
    savefig('Q2/' + filename + 'edgeMap_' + usegradient + '_' + edge_type + '.png')

    #Drawing Hough Space
    accumulator = HoughSpace(edge_map, radius, usegradient, quant)
    imshow(accumulator, cmap=cm.gray)
    title("Hough Space")
    savefig('Q2/' + filename + 'houghSpace_' + usegradient + '_' + edge_type + '_q' + str(quant) + '.png')

    # computing circle centres from local maxima of accumulator 
    centres = feature.peak_local_max(accumulator, min_distance=radius *2) #determining min_distance as diameter to avoid overlapping circles
    return centres


# this function detects and draws circles on top of an images, given the circle centre coordinates and radius    
def DrawCircles(img, centres, radius, usegradient, quant, filename, edge_type):
    h = len(img)
    w = len(img[0])
    
    # Drawing circles
    for n in range(len(centres)):
        y = centres[n][0]
        x = centres[n][1]
        yy, xx = draw.circle_perimeter(y,x,radius) # finding pixel corrdinates of circles
        for i in range(len(yy)):
            if yy[i] < h and xx[i] < w:
                img[yy[i]][xx[i]] = 1

    #saving Image with drawn circles
    imshow(img, cmap=cm.gray)
    title("Circles")
    if usegradient == "usegradient":
        savefig('Q2/' + filename + 'circles_usegradient_' + edge_type + '_q' + str(quant) + '.png')
    else:
        savefig('Q2/' + filename + 'circles_withoutgradient_' + edge_type + '_q' + str(quant) + '.png')

# this function draws the circle centres
def DrawCentres(img, centres, usegradient, quant, filename, edge_type):
    
    h = len(img)
    w = len(img[0])
    
    #drawing circle centres on a black background
    imgCentres= np.zeros((h,w))
    for n in range(len(centres)):
        b = centres[n][0]
        a = centres[n][1]
        if b < h and a < w:
            imgCentres[b][a] = 1
    
    imshow(imgCentres, cmap=cm.gray)
    title("Centres")
    if usegradient == "usegradient":
        savefig('Q2/' + filename + 'centres_usegradient_' + edge_type + '_q' + str(quant) +  '.png')
    else:
        savefig('Q2/' + filename + 'centres_withoutgradient_' + edge_type + '_q' + str(quant) + '.png')

# Using Canny, with gradient

In [4]:
if __name__ == '__main__':
    if not os.path.exists("Q2"):
        os.makedirs("Q2")
    
    images = ['Images/Q2/colorful3.jpg',
              'Images/Q2/ladybug.jpg',
              'Images/Q2/MoonCraters.jpg',
              'Images/Q2/Planets.jpeg']
    
    #Detecting Circles using Canny Edge Detector
    for image in images:
        fn = image.split('/')[-1].replace("jpg","").replace("jpeg", "")
        img = io.imread(image, as_grey=True).astype(np.float64)
        centres = DetectCircles(img, 50, "usegradient", 1, fn, "canny")
        DrawCentres(img, centres, "usegradient", 1, fn, "canny")
        DrawCircles(img, centres, 50, "usegradient", 1, fn, "canny")

# Using Canny, without gradient

In [ ]:
if __name__ == '__main__':
    if not os.path.exists("Q2"):
        os.makedirs("Q2")
    
    images = ['Images/Q2/colorful3.jpg',
              'Images/Q2/ladybug.jpg',
              'Images/Q2/MoonCraters.jpg',
              'Images/Q2/Planets.jpeg']
    
    #Detecting Circles using Canny Edge Detector
    for image in images:
        fn = image.split('/')[-1].replace("jpg","").replace("jpeg", "")
        img = io.imread(image, as_grey=True).astype(np.float64)
        centres = DetectCircles(img, 50, "withoutgradient", 1, fn, "canny")
        DrawCentres(img, centres, "withoutgradient", 1, fn, "canny")
        DrawCircles(img, centres, 50, "withoutgradient", 1, fn, "canny")

# Using Sobel, with gradient

In [6]:
if __name__ == '__main__':
    if not os.path.exists("Q2"):
        os.makedirs("Q2")
    
    images = ['Images/Q2/colorful3.jpg',
              'Images/Q2/ladybug.jpg',
              'Images/Q2/MoonCraters.jpg',
              'Images/Q2/Planets.jpeg']
    
    #Detecting Circles using Canny Edge Detector
    for image in images:
        fn = image.split('/')[-1].replace("jpg","").replace("jpeg", "")
        img = io.imread(image, as_grey=True).astype(np.float64)
        centres = DetectCircles(img, 50, "usegradient", 1,  fn, "sobel")
        DrawCentres(img, centres, "usegradient", 1, fn, "sobel")
        DrawCircles(img, centres, 50, "usegradient", 1, fn, "sobel")

# Using Sobel, without gradient

In [ ]:
if __name__ == '__main__':
    if not os.path.exists("Q2"):
        os.makedirs("Q2")
    
    images = ['Images/Q2/colorful3.jpg',
              'Images/Q2/ladybug.jpg',
              'Images/Q2/MoonCraters.jpg',
              'Images/Q2/Planets.jpeg']
    
    #Detecting Circles using Canny Edge Detector
    for image in images:
        fn = image.split('/')[-1].replace("jpg","").replace("jpeg", "")
        img = io.imread(image, as_grey=True).astype(np.float64)
        centres = DetectCircles(img, 50, "withoutgradient", 1, fn, "sobel")
        DrawCentres(img, centres, "withoutgradient", 1, fn, "sobel")
        DrawCircles(img, centres, 50, "withoutgradient", 1, fn, "sobel")

# Different Quantization

In [ ]:
if __name__ == '__main__':
    if not os.path.exists("Q2"):
        os.makedirs("Q2")

    for quant in range(1,4):
        fn = 'colorful3_'
        img = io.imread('Images/Q2/colorful3.jpg', as_grey=True).astype(np.float64)
        centres = DetectCircles(img, 50, "usegradient", quant, fn, "sobel")
        DrawCentres(img, centres, "usegradient", quant, fn, "sobel")
        DrawCircles(img, centres, 50, "usegradient", quant, fn, "sobel")